Quick mnist neural net


In [2]:
import numpy as np
import tensorflow as tf
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
from utils import shuffle_batch, variable_summaries
import os

%matplotlib notebook

dir_path = os.getcwd()

now = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)
print(dir_path)
print(now)

C:\Users\kicjo\Desktop\my-anns\CNN
2019-08-24-13-01-26


In [3]:
height = 28
width = 28
channels = 1
n_inputs = height * width

In [4]:
tf.reset_default_graph()
with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    
    y = tf.placeholder(tf.int32, shape=[None], name="y")

In [5]:
conv1 = tf.layers.conv2d(X_reshaped, filters=32, kernel_size=3,
                         strides=1, padding="SAME",
                         activation=tf.nn.relu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=64, kernel_size=3,
                         strides=2, padding="SAME",
                         activation=tf.nn.relu, name="conv2")



W0824 13:01:27.027726 14640 deprecation.py:323] From <ipython-input-5-8a99a522b569>:3: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0824 13:01:27.031684 14640 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    print(pool3.get_shape())
    pool3_flat = tf.reshape(pool3, shape=[-1, 64 * 7 * 7])
    print(pool3_flat.get_shape())

(?, 7, 7, 64)
(?, 3136)


In [7]:
with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, 64, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, 10, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

W0824 13:01:27.294978 14640 deprecation.py:323] From <ipython-input-7-81f01352886f>:2: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


In [8]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [9]:
n_epochs = 10
batch_size = 100

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Last batch accuracy:", acc_batch, "Test accuracy:", acc_test)

        save_path = saver.save(sess, "./my_mnist_model")

0 Last batch accuracy: 0.97 Test accuracy: 0.9774
1 Last batch accuracy: 0.99 Test accuracy: 0.9793
2 Last batch accuracy: 0.97 Test accuracy: 0.986
3 Last batch accuracy: 1.0 Test accuracy: 0.988
4 Last batch accuracy: 0.99 Test accuracy: 0.9874
5 Last batch accuracy: 1.0 Test accuracy: 0.9877
6 Last batch accuracy: 1.0 Test accuracy: 0.9878
7 Last batch accuracy: 1.0 Test accuracy: 0.9892
8 Last batch accuracy: 0.99 Test accuracy: 0.989
9 Last batch accuracy: 1.0 Test accuracy: 0.9897
